In [1]:
# !pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

### Only if you have a GPU

In [2]:
tf.config.list_physical_devices('GPU')

[]

In [3]:
######## GPU CONFIGS FOR RTX 2070 ###############
## Please ignore if not training on GPU       ##
## this is important for running CuDNN on GPU ##

# check if GPU can be seen by TF
tf.config.list_physical_devices('GPU')
tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
###############################################

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5



### End of GPU config section

In [3]:
# see available tfds data sets
", ".join(tfds.list_builders())

'abstract_reasoning, aeslc, aflw2k3d, amazon_us_reviews, anli, arc, bair_robot_pushing_small, beans, big_patent, bigearthnet, billsum, binarized_mnist, binary_alpha_digits, blimp, c4, caltech101, caltech_birds2010, caltech_birds2011, cars196, cassava, cats_vs_dogs, celeb_a, celeb_a_hq, cfq, chexpert, cifar10, cifar100, cifar10_1, cifar10_corrupted, citrus_leaves, cityscapes, civil_comments, clevr, clinc_oos, cmaterdb, cnn_dailymail, coco, coil100, colorectal_histology, colorectal_histology_large, common_voice, cos_e, cosmos_qa, covid19sum, crema_d, curated_breast_imaging_ddsm, cycle_gan, deep_weeds, definite_pronoun_resolution, dementiabank, diabetic_retinopathy_detection, div2k, dmlab, downsampled_imagenet, dsprites, dtd, duke_ultrasound, emnist, eraser_multi_rc, esnli, eurosat, fashion_mnist, flic, flores, food101, forest_fires, fuss, gap, geirhos_conflict_stimuli, german_credit_numeric, gigaword, glue, groove, higgs, horses_or_humans, i_naturalist2017, imagenet2012, imagenet2012_cor

In [4]:
# using TFDS dataset
# note: as_supervised converts dicts to tuples
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", 
                                with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test", 
                      as_supervised=True)

Shuffling and writing examples to /Users/ashishbansal/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKVSDDB/imdb_reviews-train.tfrecord


Shuffling and writing examples to /Users/ashishbansal/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKVSDDB/imdb_reviews-test.tfrecord


Shuffling and writing examples to /Users/ashishbansal/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKVSDDB/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /Users/ashishbansal/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
print(ds_info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [7]:
for example, label in imdb_train.take(1):
    print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


In [5]:
# Use the default tokenizer settings
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
MAX_TOKENS = 0

for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)

In [6]:
imdb_encoder = tfds.features.text.TokenTextEncoder(vocabulary_set, 
                                                   tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size

print(vocab_size, MAX_TOKENS)

93931 2525


In [7]:
# Lets verify tokenization and encoding works
for example, label in imdb_train.take(1):
    print(example)
    encoded = imdb_encoder.encode(example.numpy())
    print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This 

In [8]:
# transformation fucntions to be used with the dataset
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post', 
                                 maxlen=150)
    return np.array(pad[0], dtype=np.int64)  


def encode_tf_fn(sample, label):
    encoded = tf.py_function(encode_pad_transform, 
                                       inp=[sample], 
                                       Tout=(tf.int64))
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label

In [9]:
# test the transformation on a small subset
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)

In [10]:
for review, label in tst.take(1):
    print(review, label)
    print(imdb_encoder.decode(review))

tf.Tensor(
[  707 28104 39696 59923 80801 73651 54075 56297  8194 58848 32917 46604
 64265 19544 45456 75118 80391  9391 86931 29658 88886 22065 81938 21058
 79074  8194 66008 64621 40360 32917 51887  3640 66008 29658 57853 36090
 15074 86260 81938 73651 41235 26870 75414   707 73651 44775 39696  2905
 57857 68627 24496 84148  7111 68677 20548 43188 14484 56648 56720 76905
 11750  2733 14484 26549 66008 35157 33955 42909 56409 17906 61416 36932
 79139 14093 18598 43733 39611 48001  3856 19544 28104 25877 22065 53205
 20548 85690  4239 32917 53205 73651 50764 28104 30577 81400  8629 22542
 72235 30213 11046 17324 50764 89567 86931 11057 66147 81938 23572 11719
 41235 66147 64265 19544 41235 19130 79413 30213 36090   620 15513 60786
 14030     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [11]:
#now tokenize/encode/pad all training
# and testing data
encoded_train = imdb_train.map(encode_tf_fn,
                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn,
                             num_parallel_calls=tf.data.experimental.AUTOTUNE)

## Preparing the model

In [12]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=100

In [13]:
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model


In [14]:
model = build_model_lstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (100, None, 64)           6011584   
_________________________________________________________________
lstm (LSTM)                  (100, 64)                 33024     
_________________________________________________________________
dense (Dense)                (100, 1)                  65        
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

## Model Training and Evaluation

In [16]:
# Prefetch for performance
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [17]:
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 56s 226ms/step - loss: 0.4158 - accuracy: 0.8044 - Precision: 0.7866 - Recall: 0.8355
Epoch 2/10
250/250 [==============================] - 65s 259ms/step - loss: 0.1826 - accuracy: 0.9324 - Precision: 0.9329 - Recall: 0.9319
Epoch 3/10
250/250 [==============================] - 65s 261ms/step - loss: 0.1167 - accuracy: 0.9608 - Precision: 0.9609 - Recall: 0.9606
Epoch 4/10
250/250 [==============================] - 61s 246ms/step - loss: 0.0785 - accuracy: 0.9742 - Precision: 0.9751 - Recall: 0.9731
Epoch 5/10
250/250 [==============================] - 56s 226ms/step - loss: 0.0476 - accuracy: 0.9857 - Precision: 0.9854 - Recall: 0.9860
Epoch 6/10
250/250 [==============================] - 55s 219ms/step - loss: 0.0499 - accuracy: 0.9844 - Precision: 0.9851 - Recall: 0.9838
Epoch 7/10
250/250 [==============================] - 59s 235ms/step - loss: 0.0471 - accuracy: 0.9848 - Precision: 0.9860 - Recall: 0.9836
Epoch 8/10
250/250 [

In [18]:
model.evaluate(encoded_test.batch(BATCH_SIZE))

    250/Unknown - 20s 81ms/step - loss: 0.8282 - accuracy: 0.8362 - Precision: 0.8247 - Recall: 0.8539

[0.8282103443145752, 0.8362, 0.8246929, 0.85392]

# BiLSTM Model

In [19]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=50

In [25]:
dropout=0.2
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [26]:
bilstm = build_model_bilstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

bilstm.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (50, None, 128)           12023168  
_________________________________________________________________
dropout (Dropout)            (50, None, 128)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (50, None, 128)           98816     
_________________________________________________________________
dropout_1 (Dropout)          (50, None, 128)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (50, 128)                 98816     
_________________________________________________________________
dropout_2 (Dropout)          (50, 128)                 0         
_________________________________________________________________
dense_2 (Dense)              (50, 1)                  

In [27]:
bilstm.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

In [28]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [29]:
bilstm.fit(encoded_train_batched, epochs=5)

Epoch 1/5
500/500 [==============================] - 250s 500ms/step - loss: 0.3833 - accuracy: 0.8238 - Precision: 0.8148 - Recall: 0.8382
Epoch 2/5
500/500 [==============================] - 266s 532ms/step - loss: 0.1571 - accuracy: 0.9440 - Precision: 0.9435 - Recall: 0.9446
Epoch 3/5
500/500 [==============================] - 247s 494ms/step - loss: 0.0727 - accuracy: 0.9761 - Precision: 0.9771 - Recall: 0.9751
Epoch 4/5
500/500 [==============================] - 243s 485ms/step - loss: 0.0520 - accuracy: 0.9819 - Precision: 0.9826 - Recall: 0.9812
Epoch 5/5
500/500 [==============================] - 243s 485ms/step - loss: 0.0325 - accuracy: 0.9884 - Precision: 0.9890 - Recall: 0.9877


In [ ]:
# without dropput between layers [1.0472896423190832, 0.83548, 0.8025395, 0.88992]
bilstm.evaluate(encoded_test.batch(BATCH_SIZE))

    443/Unknown - 42s 96ms/step - loss: 0.6380 - accuracy: 0.8382 - Precision: 0.8091 - Recall: 0.8860